In [26]:
import timeit
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType

In [27]:
start_time = timeit.default_timer()

spark = SparkSession.builder \
    .appName("IcebergLocalDevelopment") \
    .master("local[*]") \
    .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2') \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()


end_time = timeit.default_timer()
elapsed_time = end_time - start_time

print("Elapsed time : ",elapsed_time)

Elapsed time :  0.006914007997693261


25/01/17 16:14:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [30]:
start_time = timeit.default_timer()

import os, json
df1=spark.read.option("multiline","true").json("amenities_sample_output.json")
df2=spark.read.option("multiline","true").json("amenity_category.json")
df3=spark.read.json("expedia-lodging-amenities-en_us-1-all.jsonl")

end_time = timeit.default_timer()
elapsed_time = end_time - start_time

print("Elapsed time : ",elapsed_time)

Elapsed time :  28.982661394002207


25/01/17 16:15:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [31]:
df3.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.sql.functions import col, concat_ws, lower, split, size

# Flatten propertyAmenities and roomAmenities
flattened_df = df3.select(
    col("propertyId.expedia").alias("expedia_id"),
    col("popularAmenities").alias("themes"),
    concat_ws(", ", *[
        col(f"propertyAmenities.{col_name}") for col_name in df3.schema["propertyAmenities"].dataType.fieldNames()
    ]).alias("property_amenities"),
    concat_ws(", ", *[
        col(f"roomAmenities.{col_name}") for col_name in df3.schema["roomAmenities"].dataType.fieldNames()
    ]).alias("room_amenities")
)

# Combine both amenities into one field
combined_df = flattened_df.select(
    col("expedia_id"),
    col("themes"),
    lower(concat_ws(", ", col("property_amenities"), col("room_amenities"))).alias("combined_amenities")
)

result_df = combined_df.withColumn(
    "amenities_count",
    size(split(col("combined_amenities"), ",\\s*"))  # Split by comma followed by optional whitespace
)

# Show the result
result_df.show(truncate=False)


+----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [37]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, lower, collect_list, size

df2_exploded = df2.select(
    explode(col("*")).alias("amenity", "category")
)

AnalysisException: [INVALID_USAGE_OF_STAR_OR_REGEX] Invalid usage of '*' in expression `explode`.